In [1]:
import pandas as pd
import json

In [40]:
with open('../data/arsonor_chunks_350_30.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [41]:
documents[372]

{'article_id': '9ca72a61',
 'url': 'https://arsonor.com/principes-de-fonctionnement-dun-synthetiseur-8-modulations-courantes/',
 'title': 'Principes de fonctionnement d’un synthétiseur (8): Modulations courantes',
 'category': 'LE SOUND DESIGN',
 'tags': 'cut-off, enveloppe ADSR, filtres, key track, LFO, modwheel, pitch, pitch bend, vélocité',
 'chunk_id': '9ca72a61-4',
 'chunk_text': "Modulation secondaire: → Sur le decay de l’enveloppe d’amplitude Permet de contrôler la longueur de notes percussives le long du clavier. Les notes plus aiguës ont un decay plus court que les notes plus graves. → Sur le LFO rate Cela peut être parfois judicieux que la hauteur de note influe sur la vitesse du LFO (vitesse de variation qui augmente avec les notes aiguës). La Vélocité Avec le KEY, la vélocité est aussi un modulateur très important dans l’objectif de s’approcher le plus possible de la sonorité et du jeu d’instruments réels. C’est un paramètre clé si tu veux « humaniser » le jeu de ton instru

## Generate Questions for each doc:

In [44]:
prompt_template = """
You emulate a beginner person in the domain of sound engineering and sound design.
Formulate 5 questions for which you will find the answer in a provided record.
These questions are in french and are questions you might generally ask in your learning path to computer music, sound design and audio post-production.
Do not refer to the document record when you ask the question.
Do not use any special characters or question mark to formulate the questions.
These questions should be complete and not too short. Use as fewer words as possible from the record. 

The record:

category: {category}
title: {title}
keywords: {tags}
answer: {chunk_text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [45]:
doc = documents[308]
prompt = prompt_template.format(**doc)
print (prompt)

You emulate a beginner person in the domain of sound engineering and sound design.
Formulate 5 questions for which you will find the answer in a provided record.
These questions are in french and are questions you might generally ask in your learning path to computer music, sound design and audio post-production.
Do not refer to the document record when you ask the question.
Do not use any special characters or question mark to formulate the questions.
These questions should be complete and not too short. Use as fewer words as possible from the record. 

The record:

category: LE HOME STUDIO
title: La gestion des niveaux sonores (1): les mesures en décibels
keywords: dB SPL, dBFS, dBm, dBu, dBV, headroom, intensité, pression, puissance, Vu-mètre
answer: Exemple de Vu-mètre sous forme de plug-in (ici celui de Waves) Cette visualisation du volume sonore est tellement pratique que l’on peut aujourd’hui retrouver ce genre d’analyseur sous forme de plugin! L’image ci-dessous représente ce v

### test on this prompt

In [18]:
from openai import OpenAI
client = OpenAI()

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [46]:
questions = llm(prompt)
json.loads(questions)

['Quelles sont les différentes unités de mesure en son et leur signification',
 'Comment fonctionne un Vu-mètre et quelle est son utilité dans un enregistrement',
 "Qu'est-ce que le headroom et pourquoi est-il important dans le mixage audio",
 'Comment le niveau moyen est-il déterminé pour différents types de signaux sonores',
 'Quelle est la différence entre le dB analogique et le dB numérique dans les enregistrements audio']

In [23]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [24]:
import time

MAX_RETRIES = 3

def generate_questions_with_retry(doc):
    for attempt in range(MAX_RETRIES):
        try:
            return generate_questions(doc)
        except Exception as e:
            print(f"Attempt {attempt + 1}/{MAX_RETRIES} failed for doc_id {doc['chunk_id']}. Retrying...")
            time.sleep(1)  # Optional: wait before retrying
    return None

In [47]:
from tqdm.auto import tqdm
results = {}

for doc in tqdm(documents):
    doc_id = doc['chunk_id']
    
    # Skip already processed documents
    if doc_id in results:
        continue

    try:
        questions_raw = generate_questions_with_retry(doc)
        
        # Try to parse the response into JSON
        questions = json.loads(questions_raw)
        results[doc_id] = questions

    except json.JSONDecodeError as e:
        # Handle JSON decoding issues and log the problematic document
        print(f"JSONDecodeError for doc_id {doc_id}: {e}")
        print(f"Problematic response: {questions_raw[:500]}")  # Print a portion of the response
        continue

    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error for doc_id {doc_id}: {e}")
        continue

  0%|          | 0/443 [00:00<?, ?it/s]

In [48]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [49]:
final_results[282]

('aa81eb34-2',
 "Quelles sont les différentes parties à prendre en compte avant d'utiliser un compresseur audio")

## Create a CSV file of the results questions:

In [50]:
doc_index = {d['chunk_id']: d for d in documents}

In [51]:
final_results = []

for doc_id, questions in results.items():
    category = doc_index[doc_id]['category']
    article_id = doc_index[doc_id]['article_id']
    for q in questions:
        final_results.append((q, category, doc_id, article_id))

In [52]:
df = pd.DataFrame(final_results, columns=['question', 'category', 'chunk', 'article'])

In [53]:
df.to_csv('ground-truth-350.csv', index=False)
pd.read_csv('ground-truth-350.csv')

,question,category,chunk,article
0,Quels sont les avantages de l'utilisation de l...,LA POST-PROD,4615db39-1,4615db39
1,Comment l'IA influence-t-elle le rôle des prod...,LA POST-PROD,4615db39-1,4615db39
2,Quelles compétences sont nécessaires pour util...,LA POST-PROD,4615db39-1,4615db39
3,Comment l'essor des home-studios a-t-il changé...,LA POST-PROD,4615db39-1,4615db39
4,Quels types de problèmes audio l'intelligence ...,LA POST-PROD,4615db39-1,4615db39
...,...,...,...,...
2210,Quels sont les éléments importants à considére...,LE SOUND DESIGN,f0da0852-14,f0da0852
2211,Comment puis-je maîtriser ma DAW avant de me c...,LE SOUND DESIGN,f0da0852-14,f0da0852
2212,Quels sont les avantages d'utiliser les plugin...,LE SOUND DESIGN,f0da0852-14,f0da0852
2213,Pourquoi est-il essentiel de ne pas se laisser...,LE SOUND DESIGN,f0da0852-14,f0da0852
